In [ ]:
!pip install scrapy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.8/287.8 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.1/247.1 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.7/97.7 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 8.7 MB/s eta 0:00:00


In [ ]:
!pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 460.2/460.2 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.7 MB/s eta 0:00:00


In [ ]:
import scrapy
from scrapy.crawler import CrawlerProcess
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait

In [ ]:
class imgtSpider(scrapy.Spider):
    name = 'new_imgt'
    start_urls = ['http://www.imgt.org/3Dstructure-DB/']



In [ ]:
 def __init__(self,hla):
        self.hla = hla
        path_to_chromedriver = '/Users/ligk2e/Downloads/chromedriver'
        self.driver = webdriver.Chrome(executable_path=path_to_chromedriver)
        self.driver.implicitly_wait(5)


In [ ]:
    def get_selenium(self,url):
        self.driver.get(url)
        self.driver.find_element_by_xpath('//*[@id="species"]/option[27]').click()
        self.driver.find_element_by_xpath('//*[@id="radio_pMH1"]').click()
        self.driver.find_element_by_xpath('//*[@id="datas"]/p[2]/input[1]').click()
        return self.driver.page_source.encode('utf-8')


In [ ]:
 def parse(self,response):
        response = scrapy.Selector(text=self.get_selenium(imgtSpider.start_urls[0]))

        for row in response.css('body#result div#data table.Results tbody tr')[1:]:
            mhc = row.css('td')[2].css('td::text').get()

            if self.hla in mhc:
                url_suffix = row.css('td')[1].css('a::attr(href)').get()
                url_next = 'http://www.imgt.org/3Dstructure-DB/cgi/' + url_suffix + '&Part=Epitope'

                yield scrapy.Request(url_next,callback=self.parse_paratope)


In [ ]:
 def parse_paratope(self,response):
        url_next = response.url
        paratope = ''
        for i in response.css('body#result div#mybody div#main table')[0].css('tr')[2].css('td')[1].css('span a'):
            aa = i.css('a::text').get()
            paratope += aa
        yield {'{}'.format(url_next):paratope}

In [ ]:
from scrapy.crawler import CrawlerProcess
from your_spider_module import imgtSpider

process = CrawlerProcess()
process.crawl(imgtSpider, hla='HLA-B*5301')
process.start()

INFO:scrapy.utils.log:Scrapy 2.11.1 started (bot: scrapybot)
2024-02-16 02:24:36 [scrapy.utils.log] INFO: Scrapy 2.11.1 started (bot: scrapybot)
INFO:scrapy.utils.log:Versions: lxml 4.9.4.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 23.10.0, Python 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0], pyOpenSSL 24.0.0 (OpenSSL 3.2.1 30 Jan 2024), cryptography 42.0.2, Platform Linux-6.1.58+-x86_64-with-glibc2.35
2024-02-16 02:24:36 [scrapy.utils.log] INFO: Versions: lxml 4.9.4.0, libxml2 2.10.3, cssselect 1.2.0, parsel 1.8.1, w3lib 2.1.2, Twisted 23.10.0, Python 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0], pyOpenSSL 24.0.0 (OpenSSL 3.2.1 30 Jan 2024), cryptography 42.0.2, Platform Linux-6.1.58+-x86_64-with-glibc2.35


ValueError: The crawler_or_spidercls argument cannot be a spider object, it must be a spider class (or a Crawler object)